## 在KITTI数据集上评估我的定位方法的效果

KITTI Object 文件目录结构

* object/
    - training/（**NOTE：本程序使用这部分数据**）
        - calib/
            - 00000.txt
            - 00001.txt
            - ...
            - 07480.txt
            
        - image_2/
            - 00000.png
            - 00001.png
            - ...
            - 07480.png
            
        - label_2/
            - 00000.txt
            - 00001.txt
            - ...
            - 07480.txt
            
    - testing/  (**NOTE: 文件夹下没有 label_2 这个目录，需要自己生成标签文件，然后上传到服务器评估**)
        - calib/
            - 00000.txt
            - 00001.txt
            - ...
            - 07510.txt
            
        - image_2/
            - 00000.png
            - 00001.png
            - ...
            - 07510.png

In [3]:
import os
import cv2
import json

### 将定位结果保存成文件

In [4]:
"""
行人3D定位部分，不涉及到 FairMOT 的代码，而是单独写的这部分代码，直接利用KITTI数据集 ——
    行人 bbox、
    行人高度、
    内参矩阵的 fx和fy、
    图片分辨率

我需要清楚的是：一个 calib_xxxxx.txt，对应一张图片 img_xxxxx.png，对应多个行人==多个行人标签
"""


data_dir = '/mnt/sdb/public/data/kitti/object/training'
label_dir = os.path.join(data_dir, 'label_2')
img_dir = os.path.join(data_dir, 'image_2')
calib_dir = os.path.join(data_dir, 'calib')

# square_dists = [100, 400, 900, 1600, 2500]
square_dists = [25, 100, 225, 400, 625, 900, 1600, 2500]
# flag2interval = {
#         0: 'd00_10',
#         1: 'd10_20',
#         2: 'd20_30',
#         3: 'd30_40',
#         4: 'd40_50',
#         5: 'd_gt_50'
#     }
flag2interval = {
        0: 'd00_05', # 2.5
        1: 'd05_10',  # 8
        2: 'd10_15',  # 12.5
        3: 'd15_20',  # 17.5
        4: 'd20_25',  # 22.5
        5: 'd25_30',  # 27.5
        6: 'd30_40',  # 35
        7: 'd40_d50',
        8: 'd_gt_50'
    }

# count = 0
for label_file in os.listdir(label_dir):
    if label_file.endswith('.txt'):
#         count += 1

        begin, end = 0, label_file.find('.')
        img_name = label_file[begin:end] + '.png'
        
        img = cv2.imread(os.path.join(img_dir, img_name))
        img_height, img_width, _ = img.shape
        
        img_x_center, img_y_center = img_width/2, img_height/2
        

#         print('calib_dir:', calib_dir)
#         print('calib_file:', calib_file)
        with open(os.path.join(label_dir, label_file)) as f_label, open(os.path.join(calib_dir, label_file)) as f_calib:
            fx = None
            fy = None
            lines = f_calib.readlines()
            for line in lines:
                if "P2:" in line:
                    values = line.strip().split(' ')
                    if len(values) > 0:
                        fx = float(values[1])
                        fy = float(values[6])
                    break
            
            
            ltrb = None
            H = None
            H_normal = None
            xyz = None
            
            """
            results存放计算ground-truth和对应的计算结果，一张图片可能有多个行人，所以要设计成这样
            
            {
                person_id: {'gt': ..., 'computed': ...}  
                ...
                ...
            }
            注：一个文件可能包含多个Pedestrian，这里person_id就是Pedestrian所在文件的行数，从0开始
            """
            results = {}
            
            lines = f_label.readlines()
            for idx, line in enumerate(lines):
                values = line.strip().split(' ')
                if len(values) > 0 and values[0] == 'Pedestrian':
                    results[idx] ={}
                    
                    # bbox of pedestrian in image plane
                    ltrb = values[4:8]
                    bbox_x_center = (float(ltrb[0]) + float(ltrb[2]))/2
                    bbox_y_center = (float(ltrb[1]) + float(ltrb[3]))/2
                    h = float(ltrb[3]) - float(ltrb[1])
                    
                    if h >= 40:
                        mode = 'easy'
                    elif h >= 25 and h < 40:
                        mode = 'moderate'
                    else:
                        mode = 'hard'
                    results[idx]['mode'] = mode
                    
                    """
                    解决单目物体尺度歧义性 --> pedestrian height: 
                    有两种选择，1）是高斯分布的均值1.7m，2）是直接从KITTI文件读取行人高度
                    注：因为我要在KITTI数据集评估，它本身提供了行人身高这个值，所以直接读取这个值进行计算
                        当数据集没有提供行人准确身高这个数值时，使用高斯分布的均值代入，这两种方法我都实验了效果，
                        差距并不大，实验证明使用准确的行人身高，定位误差会更小一些，与实际相符
                    """
#                     option 1:
                    H = float(values[8])
                    H_normal = 1.7
#                     option 2:
#                     H = 1.7

                    # ground-truth pedestrian position in camera coordiante
                    xyz = values[11:14]
                    
                    square_dist = float(xyz[0])**2 + float(xyz[1])**2 + float(xyz[2])**2
                    flag = -1
                    for i,elem in enumerate(square_dists):
                        if square_dist < elem:
                            flag = i
                            break
                    if flag == -1:
                        flag = len(square_dists)
                    interval = flag2interval[flag]
                    
                    results[idx]['interval'] = interval
                    
                    computed_x = fy/fx * H * (bbox_x_center-img_x_center) / h
                    computed_y = H * (bbox_y_center-img_y_center) / h
                    computed_z = H * fy / h
                    
                    computed_x_normal = fy/fx * H_normal * (bbox_x_center-img_x_center) / h
                    computed_y_normal = H_normal * (bbox_y_center-img_y_center) / h
                    computed_z_normal = H_normal * fy / h
                    print(label_file, '> ground-truth:', xyz[0], xyz[1], xyz[2])
#                     print(label_file, '> computed:', '{:.2f}'.format(computed_x), '{:.2f}'.format(computed_y), '{:.2f}'.format(computed_z))
                    
#                     results['gt'] = [float(xyz[0]), float(xyz[1]), float(xyz[2])]
                    results[idx]['gt'] = xyz
                    results[idx]['computed'] = ['{:.2f}'.format(computed_x), '{:.2f}'.format(computed_y), '{:.2f}'.format(computed_z)]
                    results[idx]['computed_normal'] = ['{:.2f}'.format(computed_x_normal), '{:.2f}'.format(computed_y_normal), '{:.2f}'.format(computed_z_normal)]
                                        
            if len(results) != 0:
#                 result_name = label_file[begin:end] + '.json'
                result_name = label_file[begin:end] + '.json'
                with open(os.path.join(data_dir, 'localization', result_name), 'w') as f_result:
                    json.dump(results, f_result)

006159.txt > ground-truth: -8.53 1.73 22.91
001758.txt > ground-truth: -4.29 1.51 7.41
003786.txt > ground-truth: 4.47 1.43 9.24
002885.txt > ground-truth: 5.73 1.43 19.81
007157.txt > ground-truth: 2.30 1.49 3.08
001049.txt > ground-truth: 3.05 1.49 8.49
001049.txt > ground-truth: -7.72 1.34 17.38
001049.txt > ground-truth: -6.79 1.33 17.94
001049.txt > ground-truth: -5.96 1.36 17.05
001049.txt > ground-truth: -3.36 1.47 13.52
001049.txt > ground-truth: -2.60 1.38 14.27
001049.txt > ground-truth: -6.22 1.43 14.19
001049.txt > ground-truth: -5.53 1.41 14.74
001049.txt > ground-truth: -4.97 1.38 15.24
001049.txt > ground-truth: -6.69 1.34 16.37
003502.txt > ground-truth: 1.54 0.68 22.32
003502.txt > ground-truth: -5.64 1.24 11.72
003502.txt > ground-truth: 3.05 1.60 4.71
003502.txt > ground-truth: 2.21 1.54 4.82
003502.txt > ground-truth: 4.15 1.00 18.36
003502.txt > ground-truth: 0.10 1.46 5.23
003502.txt > ground-truth: 0.38 1.51 6.21
003502.txt > ground-truth: 0.82 1.50 6.45
003502.t

004251.txt > ground-truth: 0.19 1.66 25.16
000789.txt > ground-truth: -2.39 1.51 8.28
002618.txt > ground-truth: 5.46 1.41 9.44
005784.txt > ground-truth: 4.62 1.44 15.97
002059.txt > ground-truth: -0.51 1.47 8.96
003700.txt > ground-truth: 4.57 1.40 7.68
000422.txt > ground-truth: -3.96 1.65 6.12
000422.txt > ground-truth: -4.53 1.64 6.12
000422.txt > ground-truth: 1.20 1.13 17.48
000422.txt > ground-truth: -4.36 1.19 26.51
000422.txt > ground-truth: -3.78 1.18 26.72
000422.txt > ground-truth: -7.93 1.21 23.93
005812.txt > ground-truth: -2.54 1.41 19.01
003264.txt > ground-truth: 3.02 1.64 11.76
006381.txt > ground-truth: -0.76 1.68 40.71
003703.txt > ground-truth: -2.43 1.49 9.17
004797.txt > ground-truth: 1.65 1.44 8.99
003020.txt > ground-truth: -6.33 1.62 16.03
003020.txt > ground-truth: -11.24 1.71 21.34
003020.txt > ground-truth: -5.55 1.58 26.70
006952.txt > ground-truth: 5.37 1.49 21.94
001169.txt > ground-truth: 1.08 1.37 12.99
001169.txt > ground-truth: 5.80 1.06 17.56
00116

001404.txt > ground-truth: 4.61 1.25 7.91
001404.txt > ground-truth: 1.83 1.27 38.96
001404.txt > ground-truth: 2.26 1.36 39.67
001404.txt > ground-truth: -0.75 1.30 55.29
003718.txt > ground-truth: 1.79 1.51 6.54
003718.txt > ground-truth: 4.61 1.29 8.51
003718.txt > ground-truth: 6.02 1.30 8.19
003718.txt > ground-truth: -2.21 1.41 17.35
003718.txt > ground-truth: -2.84 1.37 17.35
003718.txt > ground-truth: -0.77 1.33 19.14
003718.txt > ground-truth: -0.16 1.22 26.98
003718.txt > ground-truth: 0.55 1.27 19.00
003718.txt > ground-truth: 2.76 1.09 23.04
003718.txt > ground-truth: -3.03 1.35 20.35
003718.txt > ground-truth: 1.04 1.23 20.89
003379.txt > ground-truth: 7.43 1.46 25.56
003379.txt > ground-truth: 8.16 1.44 25.56
005696.txt > ground-truth: -3.78 1.53 5.16
005696.txt > ground-truth: -2.36 1.66 32.38
005696.txt > ground-truth: -1.66 1.51 32.08
005696.txt > ground-truth: 13.81 1.89 32.01
005696.txt > ground-truth: 14.84 2.31 31.56
002334.txt > ground-truth: -2.70 1.47 8.93
00693

000827.txt > ground-truth: -2.92 1.48 8.54
003339.txt > ground-truth: 0.70 1.53 7.05
005234.txt > ground-truth: 3.43 1.47 24.37
007052.txt > ground-truth: 16.76 1.20 50.23
004960.txt > ground-truth: 2.64 0.10 69.73
003124.txt > ground-truth: 3.21 1.56 29.02
002379.txt > ground-truth: -2.20 1.64 20.39
002379.txt > ground-truth: -2.69 1.63 20.15
002379.txt > ground-truth: 1.36 1.70 31.86
002379.txt > ground-truth: 2.21 1.76 31.79
006639.txt > ground-truth: 4.40 1.66 8.73
002327.txt > ground-truth: -3.32 1.43 17.58
002327.txt > ground-truth: -3.84 1.43 17.31
002327.txt > ground-truth: -7.54 1.21 29.05
004594.txt > ground-truth: -0.42 1.45 8.67
001979.txt > ground-truth: 4.18 1.55 15.74
001911.txt > ground-truth: -0.43 1.50 8.92
001911.txt > ground-truth: -6.81 1.50 8.85
004514.txt > ground-truth: -3.85 1.49 8.54
002493.txt > ground-truth: -1.80 1.46 8.47
003787.txt > ground-truth: 0.67 1.43 7.42
001029.txt > ground-truth: 2.27 1.47 7.66
001449.txt > ground-truth: 3.71 1.45 9.01
006256.txt

004268.txt > ground-truth: 1.88 1.46 4.77
004268.txt > ground-truth: 1.37 1.43 4.16
004268.txt > ground-truth: -1.10 1.36 6.63
004268.txt > ground-truth: -1.71 1.35 6.01
004268.txt > ground-truth: 2.34 1.46 5.22
004268.txt > ground-truth: -1.67 0.80 17.80
004268.txt > ground-truth: -2.22 0.53 21.87
004268.txt > ground-truth: -1.54 0.59 21.67
004268.txt > ground-truth: -0.49 0.55 21.25
005579.txt > ground-truth: -5.15 1.64 17.10
000339.txt > ground-truth: 5.56 1.46 7.95
006969.txt > ground-truth: 6.98 1.53 7.99
006969.txt > ground-truth: 5.72 1.66 18.50
006969.txt > ground-truth: 5.24 1.67 18.49
006932.txt > ground-truth: 2.89 1.61 5.74
005601.txt > ground-truth: -8.59 2.05 31.46
001982.txt > ground-truth: 3.77 1.42 8.99
002692.txt > ground-truth: 3.86 1.49 19.05
007414.txt > ground-truth: 4.04 1.43 15.31
003706.txt > ground-truth: -10.71 2.71 53.84
004398.txt > ground-truth: 10.15 1.35 20.36
007408.txt > ground-truth: 3.71 1.38 8.88
001267.txt > ground-truth: -5.04 1.55 9.52
001267.txt

...

001409.txt > ground-truth: 6.13 1.35 6.54
001409.txt > ground-truth: 6.29 1.42 7.19
001409.txt > ground-truth: 2.44 1.68 30.05
005208.txt > ground-truth: -4.33 2.04 38.32
006842.txt > ground-truth: -5.05 1.63 22.12
005027.txt > ground-truth: -3.83 1.44 11.73
005027.txt > ground-truth: -8.53 1.17 19.50
005027.txt > ground-truth: -10.25 1.19 15.70
005027.txt > ground-truth: -14.45 0.89 23.80
005027.txt > ground-truth: -10.61 1.14 15.82
005027.txt > ground-truth: -12.89 0.95 22.56
005027.txt > ground-truth: -13.74 0.92 22.28
005027.txt > ground-truth: -11.23 1.03 23.72
005027.txt > ground-truth: -12.22 1.00 24.33
005027.txt > ground-truth: -12.78 0.93 24.56
005027.txt > ground-truth: -13.40 0.94 24.60
005027.txt > ground-truth: -16.46 0.72 30.94
000274.txt > ground-truth: -3.21 1.97 11.22
006121.txt > ground-truth: 2.79 0.77 22.01
006121.txt > ground-truth: -5.20 1.22 11.66
006121.txt > ground-truth: 2.93 1.57 6.16
006121.txt > ground-truth: 3.10 1.40 7.14
006121.txt > ground-truth: 2.06 

002324.txt > ground-truth: 3.71 1.48 7.57
004723.txt > ground-truth: -0.55 1.44 9.73
001712.txt > ground-truth: -7.55 1.63 15.48
003250.txt > ground-truth: 5.73 1.73 23.91
004488.txt > ground-truth: -2.17 1.51 19.40
004488.txt > ground-truth: -2.02 1.63 23.87
004488.txt > ground-truth: -1.75 1.64 24.43
007182.txt > ground-truth: -4.34 1.57 15.02
007182.txt > ground-truth: -4.80 1.60 18.02
007182.txt > ground-truth: -4.45 1.59 17.52
000058.txt > ground-truth: -1.56 1.45 7.29
000532.txt > ground-truth: 2.62 1.44 9.00
006522.txt > ground-truth: 11.05 1.16 21.16
006522.txt > ground-truth: 11.52 1.08 21.42
001262.txt > ground-truth: -14.90 1.63 20.21
001262.txt > ground-truth: -15.85 1.65 28.31
005115.txt > ground-truth: 11.27 1.15 20.76
005115.txt > ground-truth: 11.81 1.09 21.04
003924.txt > ground-truth: -0.79 1.26 18.82
003924.txt > ground-truth: 0.02 1.27 18.66
003924.txt > ground-truth: -2.80 1.38 15.87
003924.txt > ground-truth: 0.10 1.45 7.68
003924.txt > ground-truth: 6.14 1.24 7.5

005824.txt > ground-truth: 3.28 1.59 5.24
005824.txt > ground-truth: 2.40 1.64 5.61
005824.txt > ground-truth: -0.29 1.77 26.62
005824.txt > ground-truth: -0.92 1.70 24.21
003146.txt > ground-truth: 5.02 1.43 22.43
003146.txt > ground-truth: 5.56 1.40 22.12
006258.txt > ground-truth: -3.45 1.49 8.70
001193.txt > ground-truth: 0.19 1.31 17.49
001193.txt > ground-truth: 0.16 1.30 16.36
001193.txt > ground-truth: 2.65 1.55 5.56
001193.txt > ground-truth: -0.47 1.37 13.18
005813.txt > ground-truth: -2.86 1.61 4.49
005813.txt > ground-truth: -3.60 1.70 14.69
005813.txt > ground-truth: -4.19 1.68 14.87
005813.txt > ground-truth: 1.52 1.75 19.18
004321.txt > ground-truth: -0.57 1.20 21.35
004321.txt > ground-truth: 0.14 1.20 21.38
004321.txt > ground-truth: -1.19 1.45 8.04
004321.txt > ground-truth: 6.15 1.26 7.57
004321.txt > ground-truth: -1.93 1.32 17.32
004321.txt > ground-truth: -0.98 1.43 10.43
004321.txt > ground-truth: 3.89 1.28 8.49
005439.txt > ground-truth: 2.73 1.49 7.41
004487.tx

005991.txt > ground-truth: -2.56 1.69 54.97
000380.txt > ground-truth: 8.44 1.58 15.11
000380.txt > ground-truth: -3.18 1.92 30.66
007077.txt > ground-truth: -2.52 1.47 8.11
005671.txt > ground-truth: -6.04 1.70 16.55
006908.txt > ground-truth: -3.24 1.66 4.27
006908.txt > ground-truth: -3.42 1.62 6.87
006908.txt > ground-truth: -4.05 1.60 6.89
006908.txt > ground-truth: 1.57 1.16 18.69
006908.txt > ground-truth: -3.51 1.26 26.91
006908.txt > ground-truth: -2.96 1.25 27.14
006908.txt > ground-truth: -7.06 1.28 24.33
001113.txt > ground-truth: 2.01 1.46 8.25
003880.txt > ground-truth: 3.92 1.34 10.32
003880.txt > ground-truth: 0.40 1.52 25.19
003880.txt > ground-truth: 1.10 1.52 25.34
003880.txt > ground-truth: -3.78 1.56 23.86
004788.txt > ground-truth: -2.96 1.09 28.83
004788.txt > ground-truth: -6.32 1.05 26.15
004788.txt > ground-truth: -7.82 1.23 27.57
007452.txt > ground-truth: -10.14 1.39 11.64
007452.txt > ground-truth: -10.12 1.42 12.28
001062.txt > ground-truth: 5.59 1.56 8.07

004524.txt > ground-truth: -8.10 1.98 38.14
002342.txt > ground-truth: -2.43 1.48 8.10
000179.txt > ground-truth: -10.77 1.86 23.51
004562.txt > ground-truth: -4.90 1.50 10.75
004562.txt > ground-truth: -0.27 1.40 15.76
004562.txt > ground-truth: -2.63 1.43 15.39
004562.txt > ground-truth: -1.49 1.35 19.32
004562.txt > ground-truth: 0.71 1.27 23.72
004562.txt > ground-truth: 0.29 1.18 30.54
004733.txt > ground-truth: -4.97 1.49 8.58
002954.txt > ground-truth: 3.40 1.59 7.34
002954.txt > ground-truth: 2.56 1.65 8.27
006262.txt > ground-truth: -3.21 1.95 24.23
001895.txt > ground-truth: 5.44 1.46 11.06
003955.txt > ground-truth: 3.54 1.59 10.50
003955.txt > ground-truth: 2.72 1.64 12.26
003763.txt > ground-truth: 1.31 1.12 15.01
003763.txt > ground-truth: -4.52 1.19 25.97
003763.txt > ground-truth: -3.91 1.20 26.13
003763.txt > ground-truth: -8.33 1.31 23.58
001357.txt > ground-truth: -1.45 1.42 11.05
001357.txt > ground-truth: -2.06 1.45 10.76
001357.txt > ground-truth: -0.10 1.42 10.12

### 评估定位结果

In [56]:
loc_dir = os.path.join(data_dir, 'localization')
def analyze_localization():
    with open(os.path.join(data_dir, 'localization', 'error_rate.json'), 'w') as fout:
        data = {}
        
        for f_name in os.listdir(loc_dir):
            if f_name.endswith('.json') and f_name != 'error_rate.json':  # 别忽略第二个条件
                f = os.path.join(loc_dir, f_name)
                with open(f) as fin:
                    results = json.load(fin)
                    data[f_name] = {}

                    for key in results.keys():
                        # convert string to float
                        gt = [ float(elem) for elem in results[key]['gt'] ]
                        computed = [ float(elem) for elem in results[key]['computed'] ]
                        # compute error
                        error = [ g-c for g,c in zip(gt, computed) ]
                        # compute error rate
                        error_rate = []
                        for e,g in zip(error, gt):
                            if g == 0:
                                error_rate.append(abs(e))
                            else:
                                error_rate.append(abs(e/g))
                        data[f_name][key] = error_rate
                        print(f_name, key, '-> error_rate:', error_rate)
                    
        json.dump(data, fout, indent=4)

In [57]:
analyze_localization()

007157.json 1 -> error_rate: [0.7260869565217394, 0.5771812080536912, 0.7272727272727273]
006301.json 7 -> error_rate: [0.023316062176165768, 0.7971014492753623, 0.02710027100271009]
006083.json 0 -> error_rate: [0.11764705882352935, 0.6518518518518519, 0.05283757338551868]
006083.json 2 -> error_rate: [0.274390243902439, 0.9150943396226415, 0.017561983471074374]
004240.json 2 -> error_rate: [0.13440860215053752, 0.7804878048780488, 0.012767066180302291]
003302.json 5 -> error_rate: [0.048739495798319335, 0.7586206896551725, 0.007727975270479108]
001371.json 6 -> error_rate: [0.12033195020746888, 0.9606741573033708, 0.010118897040222579]
003277.json 0 -> error_rate: [0.05897435897435897, 0.6643835616438356, 0.054545454545454536]
007156.json 11 -> error_rate: [0.2598425196850393, 1.2590673575129532, 0.007257291524031236]
007156.json 12 -> error_rate: [0.337313432835821, 1.2857142857142856, 0.005410523468145619]
007301.json 6 -> error_rate: [0.009389671361502356, 0.6029411764705882, 0.01

006685.json 0 -> error_rate: [0.03278688524590167, 0.6233766233766234, 0.09239940387481373]
005482.json 2 -> error_rate: [0.055693069306930715, 0.8392857142857143, 0.031353135313531275]
005482.json 3 -> error_rate: [0.059040590405904106, 0.7857142857142857, 0.029590948651000856]
001161.json 0 -> error_rate: [0.03999999999999994, 0.6688741721854304, 0.04149933065595711]
006997.json 1 -> error_rate: [0.18695652173913052, 0.5695364238410596, 0.19955654101995574]
000015.json 1 -> error_rate: [0.07999999999999997, 0.7218045112781956, 0.06587615283267458]
000015.json 2 -> error_rate: [0.16666666666666674, 0.7446808510638298, 0.021126760563380347]
000015.json 3 -> error_rate: [0.16363636363636366, 0.7714285714285714, 0.021882741535920628]
000015.json 4 -> error_rate: [0.229050279329609, 0.7867647058823529, 0.024463519313304732]
000931.json 1 -> error_rate: [0.0358514724711907, 0.8098591549295775, 0.027613412228796954]
000931.json 2 -> error_rate: [0.043580683156655003, 0.8273381294964028, 0.0

ZeroDivisionError: float division by zero

### 评估检测结果

In [ ]:
def analyze_detection():
    # 仅对有行人的图片进行检测
    for f_name in os.listdir(loc_dir):
        if f_name.endswith('.json'):
            begin, end = 0, f_name.find('.')
            img_name = f_name[begin:end]
            os.path.join()

### 用OpenCV读取照片的 宽度、长度

In [4]:
data_dir = '/mnt/sdb/public/data/kitti/object/training'
label_dir = os.path.join(data_dir, 'label_2')
img_dir = os.path.join(data_dir, 'image_2')

img = cv2.imread(os.path.join(img_dir, '000000.png'))
height, width, channels = img.shape

In [38]:
len({})

0

In [39]:
os.path.join(data_dir, 'localization', 'aaa')

'/mnt/sdb/public/data/kitti/object/training/localization/aaa'